<a href="https://colab.research.google.com/github/maha-92-no/SCD-PP/blob/main/Supply_Chain_Design_Week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week1 Practice Problem

In [ ]:
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 24.4 MB/s eta 0:00:00


In [ ]:
from gurobipy import *

## Practice Problem 2: SteelCo

In [ ]:
I,d = multidict({1:40500, 2:22230, 3:85200 , 4:47500}) # demand
J,M = multidict({1:100000, 2:100000})               # capacity
c = {(1,1):52,    (1,2):45,      # cost
     (2,1):32,    (2,2):84,
     (3,1):11,    (3,2):76,
     (4,1):69,    (4,2):15,
     }

model = Model("transportation")

# Create variables
x = {}
for i in I:
    for j in J:
        x[i,j] = model.addVar(vtype="C", name="x(%s,%s)" % (i,j))
model.update()

# Demand constraints
for i in I:
    model.addConstr(quicksum(x[i,j] for j in J if (i,j) in x) == d[i], name="Demand(%s)" % i)

# Capacity constraints
for j in J:
    model.addConstr(quicksum(x[i,j] for i in I if (i,j) in x) <= M[j], name="Capacity(%s)" % j)

# Objective
model.setObjective(quicksum(c[i,j]*x[i,j]  for (i,j) in x), GRB.MINIMIZE)

model.optimize()
print("Optimal value:", model.ObjVal)

EPS = 1.e-6
for (i,j) in x:
    if x[i,j].X > EPS:
        print("sending quantity %10s from factory %3s to customer %3s" % (x[i,j].X,j,i))


Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x4e9a0b36
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 1e+05]
Presolve removed 5 rows and 4 columns
Presolve time: 0.02s
Presolved: 1 rows, 5 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6320600e+06   6.562500e+03   0.000000e+00      0s
       1    4.5699200e+06   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.04 seconds (0.00 work units)
Optimal objective  4.569920000e+06
Optimal value: 4569920.0
sending quantity    40500.0 from factory   2 to customer   1

In [ ]:
pa_total_quantity=0
oh_total_quantity=0

for i in I:
    if x[i,1].X > EPS:
         pa_total_quantity += x[i,1].X
    if x[i,2].X > EPS:
         oh_total_quantity += x[i,2].X

print(f'pa_total_quantity:{pa_total_quantity}')
print(f'oh_total_quantity:{oh_total_quantity}')

pa_total_quantity:100000.0
oh_total_quantity:95430.0


## Practice Problem 3: Ralph Calvin Hilfiger (RCH) Industries

In [ ]:
J,M = multidict({'f1':200, 'f2':300, 'f3':100 , 'f4':150, 'f5':220}) # capacity
K = ['xd1', 'xd2']
I,d = multidict({'dc1':150, 'dc2':100, 'dc3':110 , 'dc4':200, 'dc5':180}) # demand
c = {('f1','xd1'):30,    ('f1','xd2'):50,      # cost
     ('f2','xd1'):23,    ('f2','xd2'):66,
     ('f3','xd1'):35,    ('f3','xd2'):14,
     ('f4','xd1'):70,    ('f4','xd2'):12,
     ('f5','xd1'):65,    ('f5','xd2'):70,
     ('xd1','dc1'):12,    ('xd2','dc1'):65,
     ('xd1','dc2'):25,    ('xd2','dc2'):22,
     ('xd1','dc3'):22,    ('xd2','dc3'):23,
     ('xd1','dc4'):40,    ('xd2','dc4'):12,
     ('xd1','dc5'):41,    ('xd2','dc5'):15,
     }

model = Model("transshipment problem")

# Create variables
x = {}
for k in K:
    for j in J:
        x[j,k] = model.addVar(vtype="C", name="x(%s,%s)" % (j,k))
    for i in I:
        x[k,i] = model.addVar(vtype="C", name="x(%s,%s)" % (k,i))
model.update()

# Capacity constraints
for j in J:
    model.addConstr(quicksum(x[j,k] for k in K if (j,k) in x) <= M[j], name="Capacity(%s)" % j)

# Demand constraints
for i in I:
    model.addConstr(quicksum(x[k,i] for k in K if (k,i) in x) == d[i], name="Demand(%s)" % i)

# Flow Constraints
for k in K:
    model.addConstr(quicksum(x[j,k] for j in J) == quicksum(x[k,i] for i in I))

# Objective
model.setObjective(quicksum(c[j,k]*x[j,k]  for j in J for k in K) + quicksum(c[k,i]*x[k,i] for k in K for i in I), GRB.MINIMIZE)

model.optimize()
print("Optimal value:", model.ObjVal)

EPS = 1.e-6
for j in J:
    for k in K:
        if x[j,k].X > EPS:
            print("sending quantity %10s from factory %3s to crossdocking %3s" % (x[j,k].X,j,k))

for k in K:
    for i in I:
        if x[k,i].X > EPS:
            print("sending quantity %10s from crossdocking %3s to distribution center %3s" % (x[k,i].X,k,i))

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 12 rows, 20 columns and 40 nonzeros
Model fingerprint: 0x5c6561b3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 3e+02]
Presolve removed 5 rows and 5 columns
Presolve time: 0.02s
Presolved: 7 rows, 15 columns, 30 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3800000e+03   1.600000e+02   0.000000e+00      0s
       7    3.0220000e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.04 seconds (0.00 work units)
Optimal objective  3.022000000e+04
Optimal value: 30220.0
sending quantity       60.0 from factory  f1 to crossdocking xd1
sending quantity      130.0 from factory  f1 to crossdocking xd

In [ ]:
xd2_total_quantity=0

for j in J:
    if x[j,'xd2'].X > EPS:
         xd2_total_quantity += x[j,'xd2'].X

print(f'xd2_total_quantity:{xd2_total_quantity}')

xd2_total_quantity:380.0


## Practice Problem 4: SandyCo - Multiple Optima

### Part1

In [ ]:
J,M = multidict({'p1':100, 'p2':125}) # capacity
K = ['dca', 'dcb']
I,d = multidict({'r1':25, 'r2':95, 'r3':80}) # demand
c = {('p1','dca'):190,    ('p1','dcb'):210,      # cost
     ('p2','dca'):185,    ('p2','dcb'):205,
     ('dca','r1'):175,    ('dcb','r1'):235,
     ('dca','r2'):180,    ('dcb','r2'):130,
     ('dca','r3'):165,    ('dcb','r3'):145,
     }

model = Model("transshipment problem")

# Create variables
x = {}
for k in K:
    for j in J:
        x[j,k] = model.addVar(vtype="C", name="x(%s,%s)" % (j,k))
    for i in I:
        x[k,i] = model.addVar(vtype="C", name="x(%s,%s)" % (k,i))
model.update()

model.addConstr(x['p2','dca']==105)

# Capacity constraints
for j in J:
    model.addConstr(quicksum(x[j,k] for k in K if (j,k) in x) <= M[j], name="Capacity(%s)" % j)

# Demand constraints
for i in I:
    model.addConstr(quicksum(x[k,i] for k in K if (k,i) in x) >= d[i], name="Demand(%s)" % i)

# Flow Constraints
for k in K:
    model.addConstr(quicksum(x[j,k] for j in J) == quicksum(x[k,i] for i in I))

# Objective
model.setObjective(quicksum(c[j,k]*x[j,k]  for j in J for k in K) + quicksum(c[k,i]*x[k,i] for k in K for i in I), GRB.MINIMIZE)

model.optimize()
print("Optimal value:", model.ObjVal)

EPS = 1.e-6
for j in J:
    for k in K:
        if x[j,k].X > EPS:
            print("sending quantity %10s from plant %3s to packaging center %3s" % (x[j,k].X,j,k))

for k in K:
    for i in I:
        if x[k,i].X > EPS:
            print("sending quantity %10s from packaging center %3s to customer region %3s" % (x[k,i].X,k,i))

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 10 columns and 21 nonzeros
Model fingerprint: 0x899f03c9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 1e+02]
Presolve removed 2 rows and 1 columns
Presolve time: 0.02s
Presolved: 6 rows, 9 columns, 17 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9425000e+04   3.812500e+01   0.000000e+00      0s
       5    6.9200000e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.04 seconds (0.00 work units)
Optimal objective  6.920000000e+04
Optimal value: 69200.0
sending quantity       75.0 from plant  p1 to packaging center dcb
sending quantity      105.0 from plant  p2 to packaging center 

### Part2

In [ ]:
J,M = multidict({'p1':100, 'p2':125}) # capacity
K = ['dca', 'dcb']
I,d = multidict({'r1':25, 'r2':95, 'r3':80}) # demand
c = {('p1','dca'):180,    ('p1','dcb'):210,      # cost
     ('p2','dca'):190,    ('p2','dcb'):205,
     ('dca','r1'):175,    ('dcb','r1'):235,
     ('dca','r2'):180,    ('dcb','r2'):130,
     ('dca','r3'):165,    ('dcb','r3'):145,
     }

model = Model("transshipment problem")

# Create variables
x = {}
for k in K:
    for j in J:
        x[j,k] = model.addVar(vtype="C", name="x(%s,%s)" % (j,k))
    for i in I:
        x[k,i] = model.addVar(vtype="C", name="x(%s,%s)" % (k,i))
model.update()

# Capacity constraints
for j in J:
    model.addConstr(quicksum(x[j,k] for k in K if (j,k) in x) <= M[j], name="Capacity(%s)" % j)

# Demand constraints
for i in I:
    model.addConstr(quicksum(x[k,i] for k in K if (k,i) in x) >= d[i], name="Demand(%s)" % i)

# Flow Constraints
for k in K:
    model.addConstr(quicksum(x[j,k] for j in J) == quicksum(x[k,i] for i in I))

# Objective
model.setObjective(quicksum(c[j,k]*x[j,k]  for j in J for k in K) + quicksum(c[k,i]*x[k,i] for k in K for i in I), GRB.MINIMIZE)

model.optimize()
print("Optimal value:", model.ObjVal)

EPS = 1.e-6
for j in J:
    for k in K:
        if x[j,k].X > EPS:
            print("sending quantity %10s from plant %3s to packaging center %3s" % (x[j,k].X,j,k))

for k in K:
    for i in I:
        if x[k,i].X > EPS:
            print("sending quantity %10s from packaging center %3s to customer region %3s" % (x[k,i].X,k,i))

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 10 columns and 20 nonzeros
Model fingerprint: 0x8ea11729
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 1e+02]
Presolve time: 0.05s
Presolved: 7 rows, 10 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.000000e+02   0.000000e+00      0s
       7    6.8325000e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.10 seconds (0.00 work units)
Optimal objective  6.832500000e+04
Optimal value: 68325.0
sending quantity      100.0 from plant  p1 to packaging center dca
sending quantity      100.0 from plant  p2 to packaging center dcb
sending quantity       25.0 from 

### Part3

In [ ]:
J,M = multidict({'p1':100, 'p2':125}) # capacity
K = ['dca', 'dcb']
I,d = multidict({'r1':28, 'r2':105, 'r3':87}) # demand
c = {('p1','dca'):180,    ('p1','dcb'):210,   # cost
     ('p2','dca'):190,    ('p2','dcb'):205,
     ('dca','r1'):175,    ('dcb','r1'):235,
     ('dca','r2'):180,    ('dcb','r2'):130,
     ('dca','r3'):165,    ('dcb','r3'):145,
     }

model = Model("transshipment problem")

# Create variables
x = {}
for k in K:
    for j in J:
        x[j,k] = model.addVar(vtype="C", name="x(%s,%s)" % (j,k))
    for i in I:
        x[k,i] = model.addVar(vtype="C", name="x(%s,%s)" % (k,i))
model.update()

# Capacity constraints
for j in J:
    model.addConstr(quicksum(x[j,k] for k in K if (j,k) in x) <= M[j], name="Capacity(%s)" % j)

# Demand constraints
for i in I:
    model.addConstr(quicksum(x[k,i] for k in K if (k,i) in x) >= d[i], name="Demand(%s)" % i)

# Flow Constraints
for k in K:
    model.addConstr(quicksum(x[j,k] for j in J) == quicksum(x[k,i] for i in I))

# Objective
model.setObjective(quicksum(c[j,k]*x[j,k]  for j in J for k in K) + quicksum(c[k,i]*x[k,i] for k in K for i in I), GRB.MINIMIZE)

model.optimize()
print("Optimal value:", model.ObjVal)

EPS = 1.e-6
for j in J:
    for k in K:
        if x[j,k].X > EPS:
            print("sending quantity %10s from plant %3s to packaging center %3s" % (x[j,k].X,j,k))

for k in K:
    for i in I:
        if x[k,i].X > EPS:
            print("sending quantity %10s from packaging center %3s to customer region %3s" % (x[k,i].X,k,i))

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 10 columns and 20 nonzeros
Model fingerprint: 0x63cf7155
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+02]
Presolve time: 0.03s
Presolved: 7 rows, 10 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.200000e+02   0.000000e+00      0s
       7    7.5205000e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.06 seconds (0.00 work units)
Optimal objective  7.520500000e+04
Optimal value: 75205.0
sending quantity      100.0 from plant  p1 to packaging center dca
sending quantity      120.0 from plant  p2 to packaging center dcb
sending quantity       28.0 from 

### Part4

In [ ]:
J,M = multidict({'p1':100, 'p2':125}) # capacity
K = ['dca', 'dcb']
I,d = multidict({'r1':25, 'r2':95, 'r3':80}) # demand
c = {('p1','dca'):180,    ('p1','dcb'):210,   # cost
     ('p2','dca'):190,    ('p2','dcb'):205,
     ('dca','r1'):175,    ('dcb','r1'):235,
     ('dca','r2'):180,    ('dcb','r2'):130,
     ('dca','r3'):165,    ('dcb','r3'):145,
     ('p2','r1'):275
     }

model = Model("transshipment problem")

# Create variables
x = {}
for k in K:
    for j in J:
        x[j,k] = model.addVar(vtype="C", name="x(%s,%s)" % (j,k))
    for i in I:
        x[k,i] = model.addVar(vtype="C", name="x(%s,%s)" % (k,i))

x['p2','r1'] = model.addVar(vtype="C", name="x(%s,%s)" % ('p2','r1'))
model.update()

# Capacity constraints
for j in J:
    model.addConstr(quicksum(x[j,k] for k in K if (j,k) in x) + quicksum(x[j,i] for i in I if (j,i) in x) <= M[j], name="Capacity(%s)" % j)

# Demand constraints
for i in I:
    model.addConstr(quicksum(x[k,i] for k in K if (k,i) in x) + quicksum(x[j,i] for j in J if (j,i) in x) >= d[i], name="Demand(%s)" % i)

# Flow Constraints
for k in K:
    model.addConstr(quicksum(x[j,k] for j in J) == quicksum(x[k,i] for i in I))

# Objective
model.setObjective(quicksum(c[j,k]*x[j,k]  for j in J for k in K) + quicksum(c[k,i]*x[k,i] for k in K for i in I)
                   + quicksum(x[j,i] for j in J for i in I if (j,i) in x), GRB.MINIMIZE)

model.optimize()
print("Optimal value:", model.ObjVal)

EPS = 1.e-6
for j in J:
    for k in K:
        if x[j,k].X > EPS:
            print("sending quantity %10s from plant %3s to packaging center %3s" % (x[j,k].X,j,k))

for k in K:
    for i in I:
        if x[k,i].X > EPS:
            print("sending quantity %10s from packaging center %3s to customer region %3s" % (x[k,i].X,k,i))

for j in J:
    for i in I:
        if (j,i) in x:
            if x[j,i].X > EPS:
                print("sending quantity %10s from plant %3s to customer region %3s" % (x[j,i].X,j,i))

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 11 columns and 22 nonzeros
Model fingerprint: 0x2a7a6752
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 1e+02]
Presolve time: 0.04s
Presolved: 7 rows, 11 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.000000e+02   0.000000e+00      0s
       6    5.9450000e+04   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.08 seconds (0.00 work units)
Optimal objective  5.945000000e+04
Optimal value: 59450.0
sending quantity       80.0 from plant  p1 to packaging center dca
sending quantity       95.0 from plant  p2 to packaging center dcb
sending quantity       80.0 from 

### Part5

In [ ]:
J,M = multidict({'p1':100, 'p2':125}) # capacity
K,L = multidict({'dca':1e5, 'dcb':80})
I,d = multidict({'r1':25, 'r2':95, 'r3':80}) # demand
c = {('p1','dca'):180,    ('p1','dcb'):210,   # cost
     ('p2','dca'):190,    ('p2','dcb'):205,
     ('dca','r1'):175,    ('dcb','r1'):235,
     ('dca','r2'):180,    ('dcb','r2'):130,
     ('dca','r3'):165,    ('dcb','r3'):145,
     ('p2','r1'):275
     }

model = Model("transshipment problem")

# Create variables
x = {}
for k in K:
    for j in J:
        x[j,k] = model.addVar(vtype="C", name="x(%s,%s)" % (j,k))
    for i in I:
        x[k,i] = model.addVar(vtype="C", name="x(%s,%s)" % (k,i))

x['p2','r1'] = model.addVar(vtype="C", name="x(%s,%s)" % ('p2','r1'))
model.update()

# Capacity constraints
for j in J:
    model.addConstr(quicksum(x[j,k] for k in K if (j,k) in x) + quicksum(x[j,i] for i in I if (j,i) in x) <= M[j], name="Capacity(%s)" % j)

for k in K:
    model.addConstr(quicksum(x[j,k] for j in J) + quicksum(x[k,i] for i in I) <= L[k])

# Demand constraints
for i in I:
    model.addConstr(quicksum(x[k,i] for k in K if (k,i) in x) + quicksum(x[j,i] for j in J if (j,i) in x) >= d[i], name="Demand(%s)" % i)

# Flow Constraints
for k in K:
    model.addConstr(quicksum(x[j,k] for j in J) == quicksum(x[k,i] for i in I))

# Objective
model.setObjective(quicksum(c[j,k]*x[j,k]  for j in J for k in K) + quicksum(c[k,i]*x[k,i] for k in K for i in I)
                   + quicksum(x[j,i] for j in J for i in I if (j,i) in x), GRB.MINIMIZE)

model.optimize()
print("Optimal value:", model.ObjVal)

EPS = 1.e-6
for j in J:
    for k in K:
        if x[j,k].X > EPS:
            print("sending quantity %10s from plant %3s to packaging center %3s" % (x[j,k].X,j,k))

for k in K:
    for i in I:
        if x[k,i].X > EPS:
            print("sending quantity %10s from packaging center %3s to customer region %3s" % (x[k,i].X,k,i))

for j in J:
    for i in I:
        if (j,i) in x:
            if x[j,i].X > EPS:
                print("sending quantity %10s from plant %3s to customer region %3s" % (x[j,i].X,j,i))

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 9 rows, 11 columns and 32 nonzeros
Model fingerprint: 0xcec9c473
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 1e+05]
Presolve removed 2 rows and 1 columns
Presolve time: 0.02s
Presolved: 7 rows, 10 columns, 25 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.2042200e+03   6.442775e+01   0.000000e+00      0s
       7    6.1175000e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.03 seconds (0.00 work units)
Optimal objective  6.117500000e+04
Optimal value: 61175.0
sending quantity      100.0 from plant  p1 to packaging center dca
sending quantity       35.0 from plant  p2 to packaging center

## Practice Problem 5: Food Delivery Startup

### Part1-3

In [ ]:
tw=250+150+100

X=20*250/tw + 20*150/tw + 100*100/tw
Y=30*250/tw + 40*150/tw + 150*100/tw
print(f'X={X},Y={Y}')

X=36.0,Y=57.0


In [ ]:
import numpy as np

I,x,y,w = multidict({'mit':[20,30,250/tw], 'har':[20,40,150/tw], 'yal':[100,150,100/tw]})

d=0
for i in I:
    d+=w[i]*np.sqrt((x[i]-X)**2+(y[i]-Y)**2)
d

45.27467641706711

### Part4

In [ ]:
def weber(I,x,y,w):
    """weber: model for solving the single source weber problem using soco.
    Parameters:
        - I: set of customers
        - x[i]: x position of customer i
        - y[i]: y position of customer i
        - w[i]: weight of customer i
    Returns a model, ready to be solved.
    """

    model = Model("weber")
    X,Y,z,xaux,yaux = {},{},{},{},{}
    X = model.addVar(lb=-GRB.INFINITY, vtype="C", name="X")
    Y = model.addVar(lb=-GRB.INFINITY, vtype="C", name="Y")
    for i in I:
        z[i] = model.addVar(vtype="C", name="z(%s)"%(i))
        xaux[i] = model.addVar(lb=-GRB.INFINITY, vtype="C", name="xaux(%s)"%(i))
        yaux[i] = model.addVar(lb=-GRB.INFINITY, vtype="C", name="yaux(%s)"%(i))
    model.update()

    for i in I:
        model.addConstr(xaux[i]*xaux[i] + yaux[i]*yaux[i] <= z[i]*z[i], "MinDist(%s)"%(i))
        model.addConstr(xaux[i] == (x[i]-X), "xAux(%s)"%(i))
        model.addConstr(yaux[i] == (y[i]-Y), "yAux(%s)"%(i))

    model.setObjective(quicksum(w[i]*z[i] for i in I), GRB.MINIMIZE)

    model.update()
    model.__data = X,Y,z
    return model

In [ ]:
tw=250+150+100
I,x,y,w = multidict({'mit':[20,30,250/tw], 'har':[20,40,150/tw], 'yal':[100,150,100/tw]})

#I,x,y,w = multidict({'mit':[20,30,250], 'har':[20,40,150], 'yal':[100,150,100]})

model=weber(I,x,y,w)

model.optimize()
print("Optimal value:", model.ObjVal)
print(f'X:{model.__data[0].X}, Y:{model.__data[1].X}')

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 11 columns and 12 nonzeros
Model fingerprint: 0x10862f67
Model has 3 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e-01, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Presolve removed 2 rows and 2 columns
Presolve time: 0.04s
Presolved: 4 rows, 9 columns, 8 nonzeros
Presolved model has 3 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.000e+01
 Factor Ops : 3.000e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.91170078e+02  0.0000

In [ ]:
model.__data

(<gurobi.Var X (value 19.99999997720873)>,
 <gurobi.Var Y (value 30.000001430873372)>,
 {'mit': <gurobi.Var z(mit) (value 1.56365875716335e-06)>,
  'har': <gurobi.Var z(har) (value 9.999999230744368)>,
  'yal': <gurobi.Var z(yal) (value 144.22204988926373)>})

## Practice Problem 6: Temporary Networks

In [ ]:
import pandas as pd
df=pd.read_csv('SC2x_W1PP5_Food_delivery_startup_DATA.csv')
df

,Distances in miles,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,Shelter 1,Shelter 2,Shelter 3,Shelter 4,Shelter 5,Shelter 6,Shelter 7,Shelter 8
1,DC 1,0.61,3.32,9.5,6.52,7.77,1.92,8.52,9.75
2,DC 2,6.04,0.51,1.34,6.06,0.22,6.33,4.61,3.28
3,DC 3,4.99,2.41,2.33,3.95,8.84,7.94,7.87,0.94
4,DC 4,5.58,8.8,6.32,8.54,5.15,6.06,9.42,2.16
5,DC 5,7.87,9.64,0.7,5.92,2.7,0.26,0.5,3.4
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Demand in thousands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Shelter 1,Shelter 2,Shelter 3,Shelter 4,Shelter 5,Shelter 6,Shelter 7,Shelter 8
9,NaN,14,8,25,22,20,17,18.5,23


In [ ]:
df=pd.read_csv('SC2x_W1PP5_Food_delivery_startup_DATA.csv', skiprows=1, index_col='Unnamed: 0')
df

,Shelter 1,Shelter 2,Shelter 3,Shelter 4,Shelter 5,Shelter 6,Shelter 7,Shelter 8
DC 1,0.61,3.32,9.5,6.52,7.77,1.92,8.52,9.75
DC 2,6.04,0.51,1.34,6.06,0.22,6.33,4.61,3.28
DC 3,4.99,2.41,2.33,3.95,8.84,7.94,7.87,0.94
DC 4,5.58,8.8,6.32,8.54,5.15,6.06,9.42,2.16
DC 5,7.87,9.64,0.7,5.92,2.7,0.26,0.5,3.4
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Demand in thousands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,Shelter 1,Shelter 2,Shelter 3,Shelter 4,Shelter 5,Shelter 6,Shelter 7,Shelter 8
NaN,14,8,25,22,20,17,18.5,23


In [ ]:
df_ds=df.iloc[0:5,0:8]
df_ds

,Shelter 1,Shelter 2,Shelter 3,Shelter 4,Shelter 5,Shelter 6,Shelter 7,Shelter 8
DC 1,0.61,3.32,9.5,6.52,7.77,1.92,8.52,9.75
DC 2,6.04,0.51,1.34,6.06,0.22,6.33,4.61,3.28
DC 3,4.99,2.41,2.33,3.95,8.84,7.94,7.87,0.94
DC 4,5.58,8.8,6.32,8.54,5.15,6.06,9.42,2.16
DC 5,7.87,9.64,0.7,5.92,2.7,0.26,0.5,3.4


In [ ]:
df_dm=df.iloc[-1:,0:8]
df_dm

,Shelter 1,Shelter 2,Shelter 3,Shelter 4,Shelter 5,Shelter 6,Shelter 7,Shelter 8
NaN,14,8,25,22,20,17,18.5,23


## Part1

In [ ]:
dm_dict=df_dm.to_dict(orient='records')[0]
for i in dm_dict:
    dm_dict[i]=float(dm_dict[i])*1000
dm_dict

{'Shelter 1': 14000.0,
 'Shelter 2': 8000.0,
 'Shelter 3': 25000.0,
 'Shelter 4': 22000.0,
 'Shelter 5': 20000.0,
 'Shelter 6': 17000.0,
 'Shelter 7': 18500.0,
 'Shelter 8': 23000.0}

In [ ]:
I,d = multidict(dm_dict)            # demand
J,M,f = multidict({'DC 1':[150000,100000], 'DC 2':[150000,100000], 'DC 3':[150000,100000], 'DC 4':[150000,100000], 'DC 5':[150000,100000]}) # capacity, fixed costs
c = {}   # transportation costs

for j in df_ds.columns:
    for i in df_ds.index:
        c[j,i]=float(df_ds.loc[i,j])
c

{('Shelter 1', 'DC 1'): 0.61,
 ('Shelter 1', 'DC 2'): 6.04,
 ('Shelter 1', 'DC 3'): 4.99,
 ('Shelter 1', 'DC 4'): 5.58,
 ('Shelter 1', 'DC 5'): 7.87,
 ('Shelter 2', 'DC 1'): 3.32,
 ('Shelter 2', 'DC 2'): 0.51,
 ('Shelter 2', 'DC 3'): 2.41,
 ('Shelter 2', 'DC 4'): 8.8,
 ('Shelter 2', 'DC 5'): 9.64,
 ('Shelter 3', 'DC 1'): 9.5,
 ('Shelter 3', 'DC 2'): 1.34,
 ('Shelter 3', 'DC 3'): 2.33,
 ('Shelter 3', 'DC 4'): 6.32,
 ('Shelter 3', 'DC 5'): 0.7,
 ('Shelter 4', 'DC 1'): 6.52,
 ('Shelter 4', 'DC 2'): 6.06,
 ('Shelter 4', 'DC 3'): 3.95,
 ('Shelter 4', 'DC 4'): 8.54,
 ('Shelter 4', 'DC 5'): 5.92,
 ('Shelter 5', 'DC 1'): 7.77,
 ('Shelter 5', 'DC 2'): 0.22,
 ('Shelter 5', 'DC 3'): 8.84,
 ('Shelter 5', 'DC 4'): 5.15,
 ('Shelter 5', 'DC 5'): 2.7,
 ('Shelter 6', 'DC 1'): 1.92,
 ('Shelter 6', 'DC 2'): 6.33,
 ('Shelter 6', 'DC 3'): 7.94,
 ('Shelter 6', 'DC 4'): 6.06,
 ('Shelter 6', 'DC 5'): 0.26,
 ('Shelter 7', 'DC 1'): 8.52,
 ('Shelter 7', 'DC 2'): 4.61,
 ('Shelter 7', 'DC 3'): 7.87,
 ('Shelter 7',

In [ ]:
def flp(I,J,d,M,f,c):
    """flp -- model for the capacitated facility location problem
    Parameters:
        - I: set of customers
        - J: set of facilities
        - d[i]: demand for customer i
        - M[j]: capacity of facility j
        - f[j]: fixed cost for using a facility in point j
        - c[i,j]: unit cost of servicing demand point i from facility j
    Returns a model, ready to be solved.
    """

    model = Model("flp")
    x,y = {},{}
    for j in J:
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in I:
            x[i,j] = model.addVar(vtype="C", name="x(%s,%s)"%(i,j))
    model.update()

    model.addConstr(quicksum(y[j] for j in J) == 1)

    for i in I:
        model.addConstr(quicksum(x[i,j] for j in J) == d[i], "Demand(%s)"%i)

    for j in M:
        model.addConstr(quicksum(x[i,j] for i in I) <= M[j]*y[j], "Capacity(%s)"%i)

    for (i,j) in x:
        model.addConstr(x[i,j] <= d[i]*y[j], "Strong(%s,%s)"%(i,j))

    model.setObjective(
        quicksum(f[j]*y[j] for j in J) +
        quicksum(c[i,j]*x[i,j] for i in I for j in J),
        GRB.MINIMIZE)

    model.update()
    model.__data = x,y
    return model


In [ ]:
model = flp(I,J,d,M,f,c)
model.optimize()

print("Optimal value:", model.ObjVal)

x,y = model.__data

EPS = 1.e-6
for (i,j) in x:
    if x[i,j].X > EPS:
        print("sending quantity %10s from distribution center %3s to shelter %3s" % (x[i,j].X,j,i))

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 54 rows, 45 columns and 170 nonzeros
Model fingerprint: 0x53c97e11
Variable types: 40 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [2e-01, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]
Presolve removed 54 rows and 45 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 580910 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.809100000000e+05, best bound 5.809100000000e+05, gap 0.0000%
Optimal value: 580910.0
sending quantity    14000.0 from distribution center DC 5 to shelter Shelter 1
sending quan

## Part2

In [ ]:
I,d = multidict(dm_dict)            # demand
J,M,f = multidict({'DC 1':[150000,100000], 'DC 2':[150000,100000], 'DC 3':[150000,100000], 'DC 4':[150000,100000], 'DC 5':[150000,100000]}) # capacity, fixed costs
c = {}   # transportation costs

for j in df_ds.columns:
    for i in df_ds.index:
        c[j,i]=float(df_ds.loc[i,j])*2
c

{('Shelter 1', 'DC 1'): 1.22,
 ('Shelter 1', 'DC 2'): 12.08,
 ('Shelter 1', 'DC 3'): 9.98,
 ('Shelter 1', 'DC 4'): 11.16,
 ('Shelter 1', 'DC 5'): 15.74,
 ('Shelter 2', 'DC 1'): 6.64,
 ('Shelter 2', 'DC 2'): 1.02,
 ('Shelter 2', 'DC 3'): 4.82,
 ('Shelter 2', 'DC 4'): 17.6,
 ('Shelter 2', 'DC 5'): 19.28,
 ('Shelter 3', 'DC 1'): 19.0,
 ('Shelter 3', 'DC 2'): 2.68,
 ('Shelter 3', 'DC 3'): 4.66,
 ('Shelter 3', 'DC 4'): 12.64,
 ('Shelter 3', 'DC 5'): 1.4,
 ('Shelter 4', 'DC 1'): 13.04,
 ('Shelter 4', 'DC 2'): 12.12,
 ('Shelter 4', 'DC 3'): 7.9,
 ('Shelter 4', 'DC 4'): 17.08,
 ('Shelter 4', 'DC 5'): 11.84,
 ('Shelter 5', 'DC 1'): 15.54,
 ('Shelter 5', 'DC 2'): 0.44,
 ('Shelter 5', 'DC 3'): 17.68,
 ('Shelter 5', 'DC 4'): 10.3,
 ('Shelter 5', 'DC 5'): 5.4,
 ('Shelter 6', 'DC 1'): 3.84,
 ('Shelter 6', 'DC 2'): 12.66,
 ('Shelter 6', 'DC 3'): 15.88,
 ('Shelter 6', 'DC 4'): 12.12,
 ('Shelter 6', 'DC 5'): 0.52,
 ('Shelter 7', 'DC 1'): 17.04,
 ('Shelter 7', 'DC 2'): 9.22,
 ('Shelter 7', 'DC 3'): 15.7

In [ ]:
def flp(I,J,d,M,f,c):
    """flp -- model for the capacitated facility location problem
    Parameters:
        - I: set of customers
        - J: set of facilities
        - d[i]: demand for customer i
        - M[j]: capacity of facility j
        - f[j]: fixed cost for using a facility in point j
        - c[i,j]: unit cost of servicing demand point i from facility j
    Returns a model, ready to be solved.
    """

    model = Model("flp")
    x,y = {},{}
    for j in J:
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in I:
            x[i,j] = model.addVar(vtype="C", name="x(%s,%s)"%(i,j))
    model.update()

    #model.addConstr(quicksum(y[j] for j in J) == 1)

    for i in I:
        model.addConstr(quicksum(x[i,j] for j in J) == d[i], "Demand(%s)"%i)

    for j in M:
        model.addConstr(quicksum(x[i,j] for i in I) <= M[j]*y[j], "Capacity(%s)"%i)

    for (i,j) in x:
        model.addConstr(x[i,j] <= d[i]*y[j], "Strong(%s,%s)"%(i,j))

    model.setObjective(
        quicksum(f[j]*y[j] for j in J) +
        quicksum(c[i,j]*x[i,j] for i in I for j in J),
        GRB.MINIMIZE)

    model.update()
    model.__data = x,y
    return model


In [ ]:
model = flp(I,J,d,M,f,c)
model.optimize()

print("Optimal value:", model.ObjVal)

x,y = model.__data

EPS = 1.e-6
for (i,j) in x:
    if x[i,j].X > EPS:
        print("sending quantity %10s from distribution center %3s to shelter %3s" % (x[i,j].X,j,i))

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 53 rows, 45 columns and 165 nonzeros
Model fingerprint: 0x9da9eadd
Variable types: 40 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [4e-01, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+03, 2e+04]
Presolve removed 5 rows and 0 columns
Presolve time: 0.00s
Presolved: 48 rows, 45 columns, 120 nonzeros
Variable types: 40 continuous, 5 integer (5 binary)
Found heuristic solution: objective 713420.00000

Root relaxation: cutoff, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      713420.000 713420.000  0.

## Part3

In [ ]:
I,d = multidict(dm_dict)            # demand
J,M,f = multidict({'DC 1':[150000,300000], 'DC 2':[150000,300000], 'DC 3':[150000,300000], 'DC 4':[150000,300000], 'DC 5':[150000,300000]}) # capacity, fixed costs
c = {}   # transportation costs

for j in df_ds.columns:
    for i in df_ds.index:
        c[j,i]=float(df_ds.loc[i,j])*2
c

{('Shelter 1', 'DC 1'): 1.22,
 ('Shelter 1', 'DC 2'): 12.08,
 ('Shelter 1', 'DC 3'): 9.98,
 ('Shelter 1', 'DC 4'): 11.16,
 ('Shelter 1', 'DC 5'): 15.74,
 ('Shelter 2', 'DC 1'): 6.64,
 ('Shelter 2', 'DC 2'): 1.02,
 ('Shelter 2', 'DC 3'): 4.82,
 ('Shelter 2', 'DC 4'): 17.6,
 ('Shelter 2', 'DC 5'): 19.28,
 ('Shelter 3', 'DC 1'): 19.0,
 ('Shelter 3', 'DC 2'): 2.68,
 ('Shelter 3', 'DC 3'): 4.66,
 ('Shelter 3', 'DC 4'): 12.64,
 ('Shelter 3', 'DC 5'): 1.4,
 ('Shelter 4', 'DC 1'): 13.04,
 ('Shelter 4', 'DC 2'): 12.12,
 ('Shelter 4', 'DC 3'): 7.9,
 ('Shelter 4', 'DC 4'): 17.08,
 ('Shelter 4', 'DC 5'): 11.84,
 ('Shelter 5', 'DC 1'): 15.54,
 ('Shelter 5', 'DC 2'): 0.44,
 ('Shelter 5', 'DC 3'): 17.68,
 ('Shelter 5', 'DC 4'): 10.3,
 ('Shelter 5', 'DC 5'): 5.4,
 ('Shelter 6', 'DC 1'): 3.84,
 ('Shelter 6', 'DC 2'): 12.66,
 ('Shelter 6', 'DC 3'): 15.88,
 ('Shelter 6', 'DC 4'): 12.12,
 ('Shelter 6', 'DC 5'): 0.52,
 ('Shelter 7', 'DC 1'): 17.04,
 ('Shelter 7', 'DC 2'): 9.22,
 ('Shelter 7', 'DC 3'): 15.7

In [ ]:
def flp(I,J,d,M,f,c):
    """flp -- model for the capacitated facility location problem
    Parameters:
        - I: set of customers
        - J: set of facilities
        - d[i]: demand for customer i
        - M[j]: capacity of facility j
        - f[j]: fixed cost for using a facility in point j
        - c[i,j]: unit cost of servicing demand point i from facility j
    Returns a model, ready to be solved.
    """

    model = Model("flp")
    x,y = {},{}
    for j in J:
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in I:
            x[i,j] = model.addVar(vtype="C", name="x(%s,%s)"%(i,j))
    model.update()

    #model.addConstr(quicksum(y[j] for j in J) == 1)

    for i in I:
        model.addConstr(quicksum(x[i,j] for j in J) == d[i], "Demand(%s)"%i)

    for j in M:
        model.addConstr(quicksum(x[i,j] for i in I) <= M[j]*y[j], "Capacity(%s)"%i)

    for (i,j) in x:
        model.addConstr(x[i,j] <= d[i]*y[j], "Strong(%s,%s)"%(i,j))

    model.setObjective(
        quicksum(f[j]*y[j] for j in J) +
        quicksum(c[i,j]*x[i,j] for i in I for j in J),
        GRB.MINIMIZE)

    model.update()
    model.__data = x,y
    return model


In [ ]:
model = flp(I,J,d,M,f,c)
model.optimize()

print("Optimal value:", model.ObjVal)

x,y = model.__data

EPS = 1.e-6
for (i,j) in x:
    if x[i,j].X > EPS:
        print("sending quantity %10s from distribution center %3s to shelter %3s" % (x[i,j].X,j,i))

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 53 rows, 45 columns and 165 nonzeros
Model fingerprint: 0x18be8164
Variable types: 40 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [4e-01, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+03, 2e+04]
Presolve removed 5 rows and 0 columns
Presolve time: 0.00s
Presolved: 48 rows, 45 columns, 120 nonzeros
Variable types: 40 continuous, 5 integer (5 binary)
Found heuristic solution: objective 1513420.0000

Root relaxation: objective 1.165660e+06, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1165660.00